In [1]:
from stock_env import StockEnv
import pandas as pd
import numpy as np
from replay_buffer import Buffer
from ddpg import ActorCritic
import tensorflow as tf

%load_ext autoreload
%autoreload 2

In [17]:
env_args = {
    "asset_codes": ['AAPL', 'V', 'BABA', 'ADBE', 'SNE'],
    "features": ["close", "high", "low"],
    "start_date": "2017-5-1",
    "end_date": "2017-6-23", 
    "window_len": 50,
    "data_path": "AmericaStock.csv"
}

In [18]:
stock_env = StockEnv(**env_args)

In [19]:
print(f'total number of states: {len(stock_env.states)}')

total number of states: 4


In [20]:
state_dim = tuple(stock_env.states[0].shape[1: ])
action_dim = (len(env_args['asset_codes']) + 1, )
print(f'state dimention {state_dim}')
print(f'action dimention {action_dim}')

state dimention (5, 50, 3)
action dimention (6,)


In [77]:
actor_critic = ActorCritic(state_dim, action_dim, )

Actor Network Summary: 
Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 5, 50, 3)]        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 5, 48, 2)          20        
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 5, 1, 21)          2037      
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 5, 1, 1)           22        
_________________________________________________________________
flatten_12 (Flatten)         (None, 5)                 0         
_________________________________________________________________
dense_24 (Dense)             (None, 6)                 36        
_________________________________________________________________
softmax_6 (Softmax)          (None

In [58]:
stock_env.reset()
state0 = stock_env.states[0]

In [59]:
action0 = actor_critic.main_actor(state0)
print(action0)

tf.Tensor([[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]], shape=(1, 6), dtype=float32)


In [60]:
action0 = np.squeeze(action0)

In [61]:
reward, done, state1 = stock_env.step(action0)

In [62]:
buffer_args = {
    'state_dim': state_dim,
    'action_dim': action_dim
}

In [63]:
stock_buffer = Buffer(**buffer_args)

In [64]:
stock_buffer.insert(state0, action0, reward, state1, done)

In [65]:
train_batch = stock_buffer.sample_batch()

In [66]:
actor_optimizer = tf.keras.optimizers.Adam(0.0001)
critic_optimizer = tf.keras.optimizers.Adam(0.0002)

In [67]:
actor_critic.one_step_train(train_batch, actor_optimizer, critic_optimizer)

In [68]:
actor_critic.one_step_train(train_batch, actor_optimizer, critic_optimizer)

In [73]:
train_args = {
    "env": stock_env,
    "num_eps": 20,
    "actor_lr": 0.001,
    "critic_lr": 0.0001,
    "train_every_step": 1,
    "batch_size": 1,
    "verbose": True, 
}

In [78]:
rewards = actor_critic.train(**train_args)

step 0, take action [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667], receive reward -0.00705162785070798
step 1, take action [0.1668334  0.16650015 0.16683334 0.16683276 0.16650021 0.16650015], receive reward 0.008914579997642624
step 2, take action [0.1670021  0.16633527 0.16700163 0.16698976 0.16633537 0.16633593], receive reward -0.0040519838414398
step 3, take action [0.1671703  0.16616966 0.16716854 0.16714902 0.16617005 0.1661724 ], receive reward 0.004882353797462191
Episode 0, reward: 0.002693322102957034
step 0, take action [0.16733828 0.1660037  0.16733491 0.16730955 0.16600443 0.16600913], receive reward -0.0070463351834357665
step 1, take action [0.16749965 0.16587555 0.16741373 0.16745658 0.16587964 0.16587485], receive reward 0.008921193200059643
step 2, take action [0.1676482  0.16576767 0.16743608 0.16761696 0.16577737 0.16575372], receive reward -0.004058680704200857
step 3, take action [0.16778499 0.16567826 0.16741462 0.16778576 0.16569926 0.16563